In [ ]:
import pandas as pd
from sklearn import preprocessing
from collections import deque
import random
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,LSTM,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint

In [ ]:
df=pd.read_csv('LTC-USD.csv',names=["time","low","high","open","close","volume"])
df.head()

,time,low,high,open,close,volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978


In [ ]:
print(df.shape)

(101883, 6)


In [ ]:
SEQ_LEN=60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT="LTC-USD"
EPOCHS=10
BATCH_SIZE=64
NAME=f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-pred-{int(time.time())}"

In [ ]:
main_df=pd.DataFrame()

In [ ]:
ratios=["BCH-USD","LTC-USD","ETH-USD","BTC-USD"]
for ratio in ratios:
    dataset=f"{ratio}.csv"
    df=pd.read_csv(dataset,names=["time","low","high","open","close","volume"])
    df.rename(columns={"close":f"{ratio}_close","volume":f"{ratio}_volume"},inplace=True)
    df.set_index("time",inplace=True)
    df=df[[f"{ratio}_close",f"{ratio}_volume"]]
    if len(main_df)==0:
      main_df=df
    else:
      main_df=main_df.join(df)

In [ ]:
print(main_df.head())

            BCH-USD_close  BCH-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968660     871.719971        5.675361      96.580002        9.647200   
1528968720     870.859985       26.856577      96.660004      314.387024   
1528968780     870.099976        1.124300      96.570000       77.129799   
1528968840     870.789978        1.749862      96.500000        7.216067   
1528968900     870.000000        1.680500      96.389999      524.539978   

            ETH-USD_close  ETH-USD_volume  BTC-USD_close  BTC-USD_volume  
time                                                                      
1528968660            NaN             NaN    6489.549805        0.587100  
1528968720      486.01001       26.019083    6487.379883        7.706374  
1528968780      486.00000        8.449400    6479.410156        3.088252  
1528968840      485.75000       26.994646    6479.410156        1.404100  
1528968900      4

In [ ]:
main_df['future']=main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

In [ ]:
print(main_df[['future',f"{RATIO_TO_PREDICT}_close"]])

               future  LTC-USD_close
time                                
1528968660  96.500000      96.580002
1528968720  96.389999      96.660004
1528968780  96.519997      96.570000
1528968840  96.440002      96.500000
1528968900  96.470001      96.389999
...               ...            ...
1535215020  58.080002      58.009998
1535215080  58.090000      58.020000
1535215140        NaN      58.020000
1535215200        NaN      58.080002
1535215260        NaN      58.090000

[92225 rows x 2 columns]


In [ ]:
def classify(current,future):
  if future>current:
    return 1
  else:
    return 0

In [ ]:
def preproccess_df(df):
  df.drop('future',axis=1,inplace=True)
  for col in df.columns:
    if col != "target":
      df[col]=df[col].pct_change()
      df.dropna(inplace=True)
      df[col]=preprocessing.scale(df[col].values)
  df.dropna(inplace=True)
  sequential_data=[]
  prev_days= deque(maxlen=SEQ_LEN)
  for i in df.values:
    prev_days.append([n for n in i[:-1]])
    if len(prev_days)==SEQ_LEN:
      sequential_data.append([np.array(prev_days),i[-1]])
  random.shuffle(sequential_data)
  buys=[]
  sells=[]
  for sq,target in sequential_data:
    if target==0:
      sells.append([sq,target])
    elif target==1:
      buys.append([sq,target])
  random.shuffle(buys)
  random.shuffle(sells)

  lower = min(len(buys),len(sells))
  buys=buys[:lower]
  sells=sells[:lower]

  sequential_data=buys+sells
  random.shuffle(sequential_data)

  X=[]
  Y=[]

  for sq,target in sequential_data:
    X.append(sq)
    Y.append(target)
  return np.array(X),Y

In [ ]:
main_df['target']=list(map(classify,main_df[f"{RATIO_TO_PREDICT}_close"],main_df['future']))

In [ ]:
print(main_df[['future',f"{RATIO_TO_PREDICT}_close",'target']].head())

               future  LTC-USD_close  target
time                                        
1528968660  96.500000      96.580002       0
1528968720  96.389999      96.660004       0
1528968780  96.519997      96.570000       0
1528968840  96.440002      96.500000       0
1528968900  96.470001      96.389999       1


In [ ]:
times = sorted(main_df.index.values)

In [ ]:
last_5pc=times[-int(0.05*len(times))]

In [ ]:
validation_main_df=main_df[(main_df.index)>=last_5pc]

In [ ]:
main_df=main_df[(main_df.index<last_5pc)]

In [ ]:
train_x, train_y =preproccess_df(main_df)
validation_x,validation_y=preproccess_df(validation_main_df)

<ipython-input-40-05425611cd54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('future',axis=1,inplace=True)
<ipython-input-40-05425611cd54>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col]=df[col].pct_change()
<ipython-input-40-05425611cd54>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-40-05425611cd54>:7: SettingWithCo

In [ ]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION dont Buys :{validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 69400 validation: 3688
Dont buys: 34700, buys: 34700
VALIDATION dont Buys :1844, buys: 1844


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(train_x.shape)
train_y=np.array(train_y)
validation_y=np.array(validation_y)

(69400, 60, 8)


In [ ]:
BATCH_SIZE=32
model=Sequential()
model.add(LSTM(128, activation="relu", input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, activation="relu",input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128, activation="relu",input_shape=(train_x.shape[1:]), return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2,activation="softmax"))

opt=tf.keras.optimizers.Adam(learning_rate=0.001,decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
NAME="RNN_MODEL"
log_dir = "/content/drive/MyDrive/logs/" + NAME
tensorboard = TensorBoard(log_dir=log_dir)

filepath = "/content/drive/MyDrive/models/RNN_Final-{epoch:02d}-{val_accuracy:.3f}.keras"

# Use filepath directly in ModelCheckpoint
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

history = model.fit(
    train_x,train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x,validation_y),
    callbacks=[tensorboard,checkpoint]
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/10
2169/2169 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5088 - loss: 0.7684
Epoch 1: val_accuracy improved from -inf to 0.49756, saving model to /content/drive/MyDrive/models/RNN_Final-01-0.498.keras
2169/2169 ━━━━━━━━━━━━━━━━━━━━ 61s 23ms/step - accuracy: 0.5088 - loss: 0.7684 - val_accuracy: 0.4976 - val_loss: 0.7062
Epoch 2/10
2169/2169 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5198 - loss: 0.6928
Epoch 2: val_accuracy improved from 0.49756 to 0.50217, saving model to /content/drive/MyDrive/models/RNN_Final-02-0.502.keras
2169/2169 ━━━━━━━━━━━━━━━━━━━━ 43s 20ms/step - accuracy: 0.5198 - loss: 0.6928 - val_accuracy: 0.5022 - val_loss: 0.6986
Epoch 3/10
2167/2169 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5236 - loss: 0.6921
Epoch 3: val_accuracy did not improve from 0.50217
2169/2169 ━━━━━━━━━━━━━━━━━━━━ 82s 20ms/step - accuracy: 0.5236 - loss: 0.6921 - val_accuracy: 0.4962 - val_loss: 0.7072
Epoch 4/10
2167/2169 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0